# Resultados U-Net

## Bibliotecas

In [1]:
import sys
sys.path.append('../src/') # Add src to path

In [2]:
import numpy as np
import tensorflow as tf
import pickle

In [3]:
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import shapefile as shp

In [4]:
from data.build_sequences import get_sequences
from data.processing import Scaler

## Predições com a U-Net

In [5]:
data_path = f"../data/imerg/test/*" # Dados de teste

Xtest, ytest = get_sequences(data_path, input=12, output=8)

print(f"Formato de entrada: {Xtest.shape}")
print(f"Formato de saída: {ytest.shape}")

Formato de entrada: (385, 128, 192, 12)
Formato de saída: (385, 128, 192, 8)


In [6]:
scaler_path = "../models/scaler.pkl"

with open(scaler_path, 'rb') as f:
    sc = pickle.load(f)

print(f"Média: {sc.data_mean}")
print(f"Desvio padrão: {sc.data_std}")

Média: 0.42323979220383384
Desvio padrão: 2.135493165772277


In [7]:
Xtest = sc.transform(Xtest)
ytest = sc.transform(ytest)

In [8]:
model_path = f"../models/model.h5"

model = tf.keras.models.load_model(model_path)

In [9]:
ypred = model.predict(Xtest)

13/13 [==============================] - 10s 231ms/step


In [10]:
Xtest = sc.inverse_transform(Xtest)
ytest = sc.inverse_transform(ytest)
ypred = sc.inverse_transform(ypred)

## Visualizações

In [11]:
latitude = np.round(np.loadtxt('../data/latitude.txt'), 2)
longitude = np.round(np.loadtxt('../data/longitude.txt'), 2)

In [12]:
vlat_min, vlat_max = latitude.min(), latitude.max()
vlon_min, vlon_max = longitude.min(), longitude.max()

In [13]:
i = 30
f = -1-i

max_lat, min_lat = latitude[i], latitude[f]
min_lon, max_lon = longitude[i], longitude[f]

lat_list = list(np.round(np.linspace(min_lat, max_lat, 3), 1))
lon_list = list(np.round(np.linspace(min_lon, max_lon, 5), 1))

print(f"Latitude: {lat_list}")
print(f"Longitude: {lon_list}")

Latitude: [-24.6, -28.0, -31.4]
Longitude: [-59.4, -56.2, -52.9, -49.6, -46.4]


Dados observados

In [14]:
idx = 140

frames = ytest[idx].copy()
frames[frames<=0] = np.nan

vmax = 23.5

ims = []
fig = plt.figure(figsize=(6,6))
ax = plt.axes(projection=ccrs.PlateCarree())

gl = ax.gridlines(crs=ccrs.PlateCarree(),
                  draw_labels=True,
                  linewidth=1,
                  color='black',
                  linestyle='--')

gl.top_labels = False
gl.right_labels = False
gl.xlines = True
gl.xlocator = mticker.FixedLocator(lon_list)
gl.ylocator = mticker.FixedLocator(lat_list)
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size':10, 'color':'black'}
gl.ylabel_style = {'size':10, 'color':'black'}

for i in range(frames.shape[2]):
    im = plt.imshow(frames[..., i],
                    cmap=plt.cm.rainbow,
                    vmin=0,
                    vmax=vmax,
                    extent=[vlon_min, vlon_max, vlat_min, vlat_max],
                    origin='lower',
                    animated=True)
    
    ims.append([im])

barra = np.arange(0, vmax+1, 5)

cbar = plt.colorbar(ax=ax, pad=0.02, aspect=16, shrink=0.56)
cbar.set_ticks(barra)
cbar.set_label('mm/h')

shapeID = shp.Reader("../data/shapefile/regiao_sul.shp")

for shape in shapeID.shapeRecords():
  point = np.array( shape.shape.points )
  dummy = plt.plot( point[:,0] , point[:,1], color="black", linewidth=1 )

ani = animation.ArtistAnimation(fig, ims, interval=500, blit=True, repeat_delay=1000)
ani.save(f'../reports/obs.gif', writer='pillow', fps=6)
plt.close(ani._fig)
HTML(ani.to_html5_video())

Dados preditos

In [15]:
idx = 140

frames = ypred[idx].copy()
frames[frames<=0.1] = np.nan

vmax = 23.5

ims = []
fig = plt.figure(figsize=(6,6)) # 6,6
ax = plt.axes(projection=ccrs.PlateCarree())

gl = ax.gridlines(crs=ccrs.PlateCarree(),
                  draw_labels=True,
                  linewidth=1,
                  color='black',
                  linestyle='--')

gl.top_labels = False
gl.right_labels = False
gl.xlines = True
gl.xlocator = mticker.FixedLocator(lon_list)
gl.ylocator = mticker.FixedLocator(lat_list)
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size':10, 'color':'black'}
gl.ylabel_style = {'size':10, 'color':'black'}

for i in range(frames.shape[2]):
    im = plt.imshow(frames[..., i],
                    cmap=plt.cm.rainbow,
                    vmin=0,
                    vmax=vmax,
                    extent=[vlon_min, vlon_max, vlat_min, vlat_max],
                    origin='lower',
                    animated=True)
    
    ims.append([im])

barra = np.arange(0, vmax+1, 5)

cbar = plt.colorbar(ax=ax, pad=0.02, aspect=16, shrink=0.56)
cbar.set_ticks(barra)
cbar.set_label('mm/h')

shapeID = shp.Reader("../data/shapefile/regiao_sul.shp")

for shape in shapeID.shapeRecords():
  point = np.array( shape.shape.points )
  dummy = plt.plot( point[:,0] , point[:,1], color="black", linewidth=1 )

ani = animation.ArtistAnimation(fig, ims, interval=500, blit=True, repeat_delay=1000)
ani.save(f'../reports/pred.gif', writer='pillow', fps=6)
plt.close(ani._fig)
HTML(ani.to_html5_video())